<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 4: Local Differential Privacy

Create a function that flips a virtual coin twice and randomizes data points based on the result. Call this function with incemental databases. More info about algorithm can be foud in chapter 2.3 “Formalizing differential privacy"" of Synthia Dwork book [The Algorithmic Foundations of Differential Privacy](https://github.com/AceEviliano/Reading-Facebook-Secure-and-Private-AI-Scholarship/blob/master/privacybook.pdf)

###GLOSSARY:
**Local Differential Privacy** adds noise to function data points (function inputs). Think of it as a adding noise directly to database or having individuals add noise to their own data before even putting it into database. In this settings users are **MOST protected** as they do not need to trust database owner to use their data responsibly.  <br/><br/>
**Global Differential Privacy** adds noise to function outputs, which adds noise to the output of the query on the database. That means database itself contains all of the private information. It's only the interface to the data which adds the noise necessary to ptrotect each individual's privacy.   <br/><br/>
__***Main difference between Local and Global DP***__: if the database operator is trustworthy, the only difference is that the global differential privacy leads to **more accurate results** with the same level of privacy protection. But this requires database owner to be trustworthy.  Namely, the database owner should **add noise properly**. <br/><br/>
**Trusted Curator** is an owner of a database upon which Global Differential privacy is applied. They are trusted to apply DP correctly.  <br/><br/>
Basic **sum query** is not differntially private. <br/><br/>
Differential Privacy **ALWAYS requires** a form of randomness or noise added to the query to prottect from things like Differencing Attacks. <br/><br/> 
**Randomized Response** is a technique where a certain degree of randomness can be added to the process that each individual is protected with **"plausible deniability" (flipping coin)**. Randomized Response is used in social sciences when trying to learn about the high level trends for a taboo behavior. Flipping a coin is an example of Local Differential Privacy. However **adding privacy comes at the cost of accuracy**, especially if sapmling over **a small number of people**.
<br/><br/> 
**GOALS of Differential Privacy:**
- Get the most accurate query results with the greatest amount of privacy, a.k.a. minimize the amount of noise that is added, and maximize the amount of privacy.
- Greatest fit with trust models in the actual world (don't waste trust). It looks at who trust or doesn't trust each other in a real world situation to minimize a noise accuracy tradeoff. One of the trategies there is to create create **flexible differential privacy strategies**.

In [0]:
# importing torch
import torch

In [0]:
# actual values from 150 people
db = torch.rand(150) > 0.5

In [76]:
db

tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1,
        0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1,
        1, 1, 1, 1, 0, 1], dtype=torch.uint8)

In [0]:
# mean of db with 100 people
true_result = torch.mean(db.float())

In [78]:
true_result

tensor(0.4933)

In [0]:
def query_with_and_without_noise(db):
  
  #Since we generated our db randomnly, with 50% of being 1 or 0, the mean tends to be around 0.5
  #query result over original database created
  true_data = torch.mean(db.float())
  
  # flipping two coins 100 times
  first_coin_flip = (torch.rand(len(db)) > 0.5).float()
  second_coin_flip = (torch.rand(len(db)) > 0.5).float()
  
  # will return 1 only for the places in the database where there actually was a 1 originally 
  # db.float() * first_coin_flip
  
  # augmented_database is differentially private !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  augmented_database = db.float() * first_coin_flip + (1-first_coin_flip) * second_coin_flip
  # # (1-first_coin_flip) are all the places we want to choose randomly
  # torch.mean(augmented_database.float())  skewd results
  
  # deskew results = TRUE OUTPUT OF THE DATABASE !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  with_noise_result = torch.mean(augmented_database.float()) *2 - 0.5
  
  return true_data, with_noise_result

In [80]:
# db size 10
db = torch.rand(10) > 0.5
with_noise_result, true_data = query_with_and_without_noise(db)
print("Without Noise:" + str(true_data))
print("With Noise:" + str(with_noise_result))

Without Noise:tensor(0.5000)
With Noise:tensor(0.7000)


In [81]:
# db size 100
db = torch.rand(100) > 0.5
with_noise_result, true_data = query_with_and_without_noise(db)
print("Without Noise:" + str(true_data))
print("With Noise:" + str(with_noise_result))

Without Noise:tensor(0.4800)
With Noise:tensor(0.4400)


In [82]:
# db size 1000
db = torch.rand(1000) > 0.5
with_noise_result, true_data = query_with_and_without_noise(db)
print("Without Noise:" + str(true_data))
print("With Noise:" + str(with_noise_result))


Without Noise:tensor(0.4680)
With Noise:tensor(0.5060)


In [83]:
# db size 10000
db = torch.rand(10000) > 0.5
with_noise_result, true_data = query_with_and_without_noise(db)
print("Without Noise:" + str(true_data))
print("With Noise:" + str(with_noise_result))

Without Noise:tensor(0.5110)
With Noise:tensor(0.5073)
